# RAG intro

<img src="images/rag_base.png" width=800px />

## 1. Embeddings and Vector DB

<img src="images/rag_vectors.png" width=800px />

In [ ]:
# pip install pdfplumber
# pip install chromadb


In [ ]:
# create storage

import os


from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PDFPlumberLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# Define the directory containing the text file and the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "texts", "Мастер и Маргарита.pdf")
persistent_directory = os.path.join(current_dir, "db", "chroma_db")

# Check if the Chroma vector store already exists
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    # Ensure the text file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"The file {file_path} does not exist. Please check the path."
        )

    # Read the text content from the file
    loader = PDFPlumberLoader(file_path)
    documents = loader.load()
    text = " ".join([document.page_content for document in documents])

    documents = [Document(
        page_content=text,
        metadata={"source": "Мастер и Маргарита.pdf"}
    )]

    # Split the document into chunks
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100, separator='\n')
    docs = text_splitter.split_documents(documents)
    # docs = text_splitter.split_text(text)

    # Display information about the split documents
    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[1].page_content}\n")
    # print(f"Sample chunk:\n{docs[1]}\n")

    # Create embeddings
    print("\n--- Creating embeddings ---")
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )  # Update to a valid embedding model if needed
    print("\n--- Finished creating embeddings ---")

    # Create the vector store and persist it automatically
    print("\n--- Creating vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)

    # db = Chroma.from_texts(docs, embeddings, persist_directory=persistent_directory)
    print("\n--- Finished creating vector store ---")

else:
    print("Vector store already exists. No need to initialize.")

Persistent directory does not exist. Initializing vector store...

--- Document Chunks Information ---
Number of document chunks: 1862
Sample chunk:
глава II. Понтий Пилат / 23
глава III. седьмое доказательство / 49
глава IV. Погоня / 55
глава V. Было дело в грибоедове / 63
глава VI. Шизофрения, как и было сказано / 77
глава VII. Нехорошая квартирка / 87
глава VIII. Поединок между профессором и поэтом / 99
глава IX. Коровьевские штуки / 109
глава X. Вести из Ялты / 119
глава XI. раздвоение ивана / 131
глава XII. Черная магия и ее разоблачение / 135
глава XIII. Явление героя / 151
глава XIV. слава петуху! / 173


--- Creating embeddings ---

--- Finished creating embeddings ---

--- Creating vector store ---

--- Finished creating vector store ---


In [ ]:
# Define the user's question
query = "В каком городе оказался Лиходеев?"


# Retrieve relevant documents based on the query
# retriever = db.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={"k": 3, "score_threshold": 0.2},
# )
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
# search_type="mmr"

relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
ставили к нему вооруженную охрану. из Москвы телеграммой
было приказано римского под охраной доставить в Москву,
вследствие чего римский в пятницу вечером и выехал под такой
охраной с вечерним поездом.
К вечеру же пятницы нашли и след лиходеева. Во все горо-
да были разосланы телеграммы с запросами о лиходееве, и из
Ялты был получен ответ, что лиходеев был в Ялте, но вылетел
на аэроплане в Москву.
единственно, чей след не удалось поймать, это след Варенухи.

Source: Мастер и Маргарита.pdf

Document 2:
потому что на вопросы, где лиходеев, Варенуха, римский,
отвечать было решительно нечего. сперва пробовали отде-
латься словами «лиходеев на квартире», а из города отвечали,
что звонили на квартиру и что квартира говорит, что лиходеев
в Варьете.
Позвонила взволнованная дама, стала требовать римского,
ей посоветовали позвонить к жене его, на что трубка, зары-
дав, ответила, что она и есть жена и что римского нигде нет.
Начиналась какая-то чепуха. Убор

In [5]:
import os

from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)

db_dir = os.path.join(current_dir, "db")

def create_vector_store(docs, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- Creating vector store {store_name} ---")
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        print(f"--- Finished creating vector store {store_name} ---")
    else:
        print(
            f"Vector store {store_name} already exists. No need to initialize.")

# 1. Character-based Splitting
# Splits text into chunks based on a specified number of characters.
# Useful for consistent chunk sizes regardless of content structure.
print("\n--- Using Character-based Splitting ---")
char_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator='\n')
char_docs = char_splitter.split_documents(documents)
create_vector_store(char_docs, "chroma_db_char")

# 2. Sentence-based Splitting
# Splits text into chunks based on sentences, ensuring chunks end at sentence boundaries.
# Ideal for maintaining semantic coherence within chunks.
print("\n--- Using Sentence-based Splitting ---")
sent_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1000)
sent_docs = sent_splitter.split_documents(documents)
create_vector_store(sent_docs, "chroma_db_sent")

# 3. Token-based Splitting
# Splits text into chunks based on tokens (words or subwords), using tokenizers like GPT-2.
# Useful for transformer models with strict token limits.
print("\n--- Using Token-based Splitting ---")
token_splitter = TokenTextSplitter(chunk_overlap=0, chunk_size=512)
token_docs = token_splitter.split_documents(documents)
create_vector_store(token_docs, "chroma_db_token")

# 4. Recursive Character-based Splitting
# Attempts to split text at natural boundaries (sentences, paragraphs) within character limit.
# Balances between maintaining coherence and adhering to character limits.
print("\n--- Using Recursive Character-based Splitting ---")
rec_char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separators=['\n', '\n '])
rec_char_docs = rec_char_splitter.split_documents(documents)
create_vector_store(rec_char_docs, "chroma_db_rec_char")

# 5. Custom Splitting
# Allows creating custom splitting logic based on specific requirements.
# Useful for documents with unique structure that standard splitters can't handle.
print("\n--- Using Custom Splitting ---")


class CustomTextSplitter(TextSplitter):
    def split_text(self, text):
        # Custom logic for splitting text
        return text.split("\n\n")  # Example: split by paragraphs


custom_splitter = CustomTextSplitter()
custom_docs = custom_splitter.split_documents(documents)
create_vector_store(custom_docs, "chroma_db_custom")


# Function to query a vector store
def query_vector_store(store_name, query):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n--- Querying the Vector Store {store_name} ---")
        db = Chroma(
            persist_directory=persistent_directory, embedding_function=embeddings
        )
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 1, "score_threshold": 0.1},
        )
        relevant_docs = retriever.invoke(query)
        # Display the relevant results with metadata
        print(f"\n--- Relevant Documents for {store_name} ---")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"Document {i}:\n{doc.page_content}\n")
            if doc.metadata:
                print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        print(f"Vector store {store_name} does not exist.")


# Define the user's question
query = "В какой квартире жил Воланд в Москве"

# Query each vector store
query_vector_store("chroma_db_char", query)
query_vector_store("chroma_db_sent", query)
query_vector_store("chroma_db_token", query)
query_vector_store("chroma_db_rec_char", query)
query_vector_store("chroma_db_custom", query)


--- Using Character-based Splitting ---
Vector store chroma_db_char already exists. No need to initialize.

--- Using Sentence-based Splitting ---


/opt/anaconda3/envs/llm-intro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



--- Creating vector store chroma_db_sent ---
--- Finished creating vector store chroma_db_sent ---

--- Using Token-based Splitting ---

--- Creating vector store chroma_db_token ---
--- Finished creating vector store chroma_db_token ---

--- Using Recursive Character-based Splitting ---

--- Creating vector store chroma_db_rec_char ---
--- Finished creating vector store chroma_db_rec_char ---

--- Using Custom Splitting ---

--- Creating vector store chroma_db_custom ---
--- Finished creating vector store chroma_db_custom ---

--- Querying the Vector Store chroma_db_char ---


/var/folders/ck/mnjcv5_14nq5jzl5wb8kd5gh0000gn/T/ipykernel_31392/1343586328.py:80: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(



--- Relevant Documents for chroma_db_char ---
Document 1:
Могарыч. словом, знакомство с Воландом не принесло ей
никакого психического ущерба. Все было так, как будто так
и должно быть. Она пошла в соседнюю комнату, убедилась
в том, что мастер спит крепким и спокойным сном, погасила
ненужную настольную лампу и сама протянулась под про-
тивоположной стеной на диванчике, покрытом старой разо-
рванной простыней. Через минуту она спала, и никаких снов
в то утро она не видела. Молчали комнаты в подвале, молчал
весь маленький домишко застройщика, и тихо было в глухом
переулке.
Но в это время, то есть на рассвете субботы, не спал целый
этаж в одном из московских учреждений, и окна в нем, вы-
ходящие на залитую асфальтом большую площадь, которую
специальные машины, медленно разъезжая с гудением, чис-
тили щетками, светились полным светом, прорезавшим свет
восходящего солнца.
Весь этаж был занят следствием по делу Воланда, и лампы
всю ночь горели в десяти кабинетах.
377
 собственно говоря, дело

/opt/anaconda3/envs/llm-intro/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'Мастер и Маргарита.pdf'}, page_content='Мастер и Маргарита\n \n Мастер\nи\nМаргарита\nМихаил Булгаков\n Москва 1984 г.\nТекст печатается в последней прижизнен-\nной редакции (рукописи хранятся в руко-\nписном отделе Государственной библио-\nтеки СССР имени В. И. Ленина), а также\nс исправлениями и дополнениями, сделан-\nными под диктовку писателя его женой,\nЕ. С. Булгаковой.\n Содержание\nглава I. Никогда не разговаривайте с неизвестными / 9\nглава II. Понтий Пилат / 23\nглава III. седьмое доказательство / 49\nглава IV. Погоня / 55\nглава V. Было дело в грибоедове / 63\nглава VI. Шизофрения, как и было сказано / 77\nглава VII. Нехорошая квартирка / 87\nглава VIII. Поединок между профессором и поэтом / 99\nглава IX. Коровьевские штуки / 109\nглава X. Вести из Ялты / 119\nглава XI. раздвоение ивана


--- Relevant Documents for chroma_db_custom ---


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import chain
from langchain_core.documents import Document

from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv(override=True)

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o", verbose=True)

# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
prompt.pretty_print()

@chain
def retriever_score(query: str) -> List[Document]:
    docs, scores = zip(*db.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score
        print(score)

    return docs

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever_score | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("Какой номер квартиры где жил воланд?")


/opt/anaconda3/envs/llm-intro/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
1.0296753644943237
1.0951279401779175
1.1140828132629395
1.1252774000167847


'Воланд жил в квартире Степана Богдановича Лиходеева.'

<img src="images/rag_type_query.png" width=800px />

## 3. RAG baseline

<img src="images/rag_retrive.png" width=800px />

# 4. Advanced RAG approaches

## 5. GraphRAG